In [1]:
import warnings
warnings.filterwarnings('ignore')
import gc

In [2]:
import matplotlib.pylab as plt
import numpy as np
from mgemu import emu
import pyccl

In [3]:
def pmg(Om, h, ns, s8, fR0, n, z):
    Omh2 = (h**2)*Om
    pkratio, k = emu(Omh2, ns, s8, fR0, n, z)
    gc.collect()
    # cosmo = pyccl.Cosmology(Omega_c= Om, Omega_b=0.0, h=h, n_s=ns, sigma8=s8, transfer_function='bbks')
    cosmo = pyccl.Cosmology(Omega_c=Om-0.02203/(h**2), Omega_b=0.02203/(h**2), h=h, sigma8=s8, n_s=ns, Neff=3.04, transfer_function='boltzmann_class', matter_power_spectrum='emu')
    a = 1./(1+z)
    #Now we have to be careful, because mgemu k-units are in h/Mpc, while CCL units are in 1/Mpc. convert
    kccl = k*h
    pk_nl = pyccl.nonlin_matter_power(cosmo, kccl, a)
    #CCL output pk is in Mpc^3, convert to (Mpc/h)^3
    pk_nl *= h*h*h
    #pk_mg = pk_lcdm*pkratio
    pk_mg = pk_nl*pkratio
    return pk_mg, k

In [4]:
#Defining function to calculate galaxy power spectrum, using the matter Pk from function pmg
#Additional arguments needed are linear bias b1& snot noise term shot
def p_ggmg(Om, h, ns, s8, fR0, n, z, b1, shot):
    Omh2 = (h**2)*Om
    pkratio, k = emu(Omh2, ns, s8, fR0, n, z)
    gc.collect()
    cosmo = pyccl.Cosmology(Omega_c=Om-0.02203/(h**2), Omega_b=0.02203/(h**2), h=h, sigma8=s8, n_s=ns, Neff=3.04, transfer_function='boltzmann_class', matter_power_spectrum='emu')
    a = 1./(1+z)
    #Now we have to be careful, because mgemu k-units are in h/Mpc, while CCL units are in 1/Mpc. convert k-emu range
    kccl = k*h
    pk_nl = pyccl.nonlin_matter_power(cosmo, kccl, a)
    #CCL output pk is in Mpc^3, convert to (Mpc/h)^3
    pk_nl *= h*h*h
    #pk_mg = pk_lcdm*pkratio
    pk_mg = pk_nl*pkratio
    pgg_mg = b1*b1*pk_mg + shot
    return pgg_mg, k

In [5]:
### LCDM parameters
h=0.6774 # See README and the accompanying paper regarding the value of h. 
Om = 0.3089
Omh2=(h**2)*Om
Obh2 = 0.02203
Och2 = Omh2-Obh2
ns=0.9667
s8=0.8159
### Hu-Sawicki model parameters
fr0=1e-5
n=1
### Redshift
z=0

In [6]:
np.random.randint(1e5, size=1)

array([65760])

In [7]:
pkratio , k = emu(Omh2=Omh2, ns=ns, s8=s8, fR0=fr0, n=n, z=0.031)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
This op will be removed after the deprecation date. Please switch to tf.sets.difference().


Instructions for updating:
This op will be removed after the deprecation date. Please switch to tf.sets.difference().


In [8]:
# pkratio

In [9]:
# cosmo = pyccl.Cosmology(Omega_c= Om, Omega_b=0.0, h=h, n_s=ns, sigma8=s8, transfer_function='bbks')
cosmo = pyccl.Cosmology(Omega_c=0.26-0.0453, Omega_b=0.0453, h=0.72, sigma8=0.80, n_s=0.96, Neff=3.04, transfer_function='boltzmann_class', matter_power_spectrum='emu')
#pyccl.comoving_radial_distance(cosmo, 1./(1+z))
a = 1./(1+z)
pk_lcdm = pyccl.nonlin_matter_power(cosmo, k*h, a)
pk_lcdm *= h*h*h

In [10]:
#Now that we can evalute the galaxy Pk for a given MG cosmology, let's build functions to actually predict the galaxy bias and shot noise for a realistic LSST sample 

In [11]:
import math
from scipy.interpolate import interp1d
from scipy.integrate import quad, dblquad
from scipy.special import gamma
from scipy.integrate import odeint

In [12]:
#Define a-dependent functions for growth factor equation
def Omegatime(a, Om0):
 return Om0/(Om0+(1-Om0)*a*a*a)
#adot
def adot(a, Om0):
 return np.sqrt(Om0/a+(1-Om0)*a*a)
def H(a, Om0):
 return adot(a, Om0)/a
def adotprime(a, Om0):
 return (-Om0/a/a + 2*(1-Om0)*a)/np.sqrt(Om0/a+(1-Om0)*a*a)/2
#Define additional functions needed for
def mfr(a, Om0, fr0, nfr): #scalar field mass
 return (1/2997.72)*math.sqrt(1./(nfr+1)/fr0)*math.sqrt(math.pow(Om0+4*(1-Om0),-nfr-1))*math.sqrt(math.pow(Om0/a/a/a+4*(1-Om0),2+nfr))
#Define 1/adot^3 integrand
def invadot3(a, Om0):
 return 1/adot(a, Om0)/adot(a, Om0)/adot(a, Om0)
#Define 1/adot^3 integral
def intToday(Om0, amin):
 return quad(invadot3,0,amin,args=(Om0))[0]
def Damin(Om0, amin):
 return 2.5*Om0*H(amin, Om0)*intToday(Om0, amin)
def DH1(a, Om0):
 return -3*Om0/(2*a*a*a*math.sqrt((a*a*a+Om0-Om0*a*a*a)/a))
def der0(Om0, amin):
 return 2.5*Om0*(DH1(amin, Om0)*intToday(Om0, amin)+H(amin, Om0)*invadot3(amin, Om0))

In [13]:
#Setup integration to get comoving volume for various z bin
#Comoving distance integrator
def integrand(a, Omm):
 return 1/math.sqrt(a*Omm+a*a*a*a*(1-Omm))
#get comoving volume
def Vcom(z, Omm):
 return (4*math.pi*20000)/(3*41252.96)*math.pow(2997.92458*quad(integrand, 1./(1+z), 1, args=(Omm))[0],3)#*math.pow(10,-9)
#get comoving differential volume in redshift bin with width +-0.05
def Vbin(z, Omm, bin):
 return Vcom(z+bin, Omm)-Vcom(z-bin, Omm)
Veff=np.vectorize(Vbin)

#Define function fro differential dN/dz to be integrated
def dNdz(z, alpha, z0, ntot):
 return (ntot*alpha/(z0*gamma(3.0/alpha)))*(z/z0)*(z/z0)*math.exp(-math.pow(z/z0,alpha))
def Nz(z, alpha, z0, ntot, bin):
 return quad(dNdz, z-bin, z+bin, args=(alpha, z0, ntot))[0]

In [14]:
#Setup basic parameters for year10&Y1 LSST photo-z samples
NY10=48*20000*3600
ay10= 0.90
z0y10 = 0.28
biny10 = 0.05
b1y10 = 0.95
#for year1 sample
NY1=18*20000*3600
ay1= 0.94
z0y1 = 0.26
biny1 = 0.1
b1y1 = 1.05

In [15]:
abserr = 1.0e-13
relerr = 1.0e-13
#Define growth factor differential equation system
def growth(y, a, Om0):
 D, w = y
 dyda = [w, -(adotprime(a, Om0) + 2*H(a, Om0))*w/adot(a, Om0) + 1.5*Omegatime(a, Om0)*H(a, Om0)*H(a, Om0)*D/adot(a, Om0)/adot(a, Om0)]
 return dyda
 
#y0 = [0.0019999999,0.999999999]
arange = np.logspace(math.log(0.002,10),math.log(1,10),1000)
#setting up boundary conditions
y0 = [Damin(Om, arange[0]),der0(Om, arange[0])]
sol = odeint(growth, y0, arange, args=(Om,), atol=abserr, rtol=relerr)
#Spline GR growth factor  solution and growth rate.
Dgr = interp1d(arange, sol[:,0], kind='cubic')
#and also growth rate
fgr = interp1d(arange, arange*sol[:,1]/sol[:,0], kind='cubic')

In [16]:
timesteps = np.loadtxt('timestepsCOLA.txt')
irangey10 = np.array([43,45,48,53,57,61,65,70,75,82])
irangey1 = np.array([43,48,57,65,75])
Nt = irangey10[irangey10.shape[0]-1]

Nseed=2000 #25

In [17]:
#Now iterate of different snapshots
for i in (irangey10):
 astep = timesteps[i,0]
 zstep = timesteps[i,1]
 #linear bias for galaxy sample, in our case Y10
 b1=b1y10*Dgr(1)/Dgr(astep) #Bias for Y10
 #b1=b1y1*Dgr(1)/Dgr(astep) #Bias for Y10
 print ('a,z,b_1=',astep, zstep, b1)

a,z,b_1= 0.4512 1.216313 1.7096583337348537
a,z,b_1= 0.4708 1.124045 1.645654696216662
a,z,b_1= 0.5002 0.999201 1.5599659653732123
a,z,b_1= 0.5492 0.820831 1.439767025161971
a,z,b_1= 0.5884 0.699524 1.359885641811489
a,z,b_1= 0.6276 0.593372 1.2915087247085706
a,z,b_1= 0.6668 0.4997 1.232567867300583
a,z,b_1= 0.7158 0.397038 1.169726499808988
a,z,b_1= 0.7648 0.307531 1.1166678302825346
a,z,b_1= 0.8334 0.199904 1.0553364364635174


In [18]:
#print (Nseed, k.shape[0],((Nseed-k.shape[0]-2)/(Nseed-1)))
#Import covariance matrices
for i in (irangey10):
    astep = timesteps[i,0]
    zstep = timesteps[i,1]
    locals()['covPMGmat_'+str(i)] = np.loadtxt('./Covariance_data/covariance_step_'+str(i)+'.txt')
    locals()['InvcovPMGmat_'+str(i)] = ((Nseed-k.shape[0]-2)/(Nseed-1))*np.linalg.inv(locals()['covPMGmat_'+str(i)])
    

In [19]:
for i in (irangey10):
    astep = timesteps[i,0]
    zstep = timesteps[i,1]
    print (zstep)

1.216313
1.124045
0.999201
0.820831
0.699524
0.593372
0.4997
0.397038
0.307531
0.199904


In [20]:
npar = 6 #no of paramaters to constrain
FY10tot = np.zeros([npar,npar])
FY1tot = np.zeros([npar,npar])
imax =  47 #135 #maximum k scale array index
#imax = 21

In [21]:
nn=1

In [22]:
from matplotlib import gridspec
from matplotlib import cm

In [23]:

import time

In [24]:
#New part to test Nesar's Fisher code
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
from scipy.misc import derivative




def plot_contours(hessian, pos,  nstd=1., ax=None, **kwargs):
  """
  Plot 2D parameter contours given a Hessian matrix of the likelihood
  """
  
  def eigsorted(cov):
    vals, vecs = np.linalg.eigh(cov)
    order = vals.argsort()[::-1]
    return vals[order], vecs[:, order]

  mat = - hessian

  cov = np.linalg.pinv(mat)

  sigma_marg = lambda i: np.sqrt(cov[i, i])

  if ax is None:
      ax = plt.gca()

  vals, vecs = eigsorted(cov)
  theta = np.degrees(np.arctan2(*vecs[:, 0][::-1]))

  # Width and height are "full" widths, not radius
  width, height = 2 * nstd * np.sqrt(vals)
  ellip = Ellipse(xy=pos, width=width,
                  height=height, angle=theta, **kwargs)

  ax.add_artist(ellip)
  sz = max(width, height)
  s1 = 1.5*nstd*sigma_marg(0)
  s2 = 1.5*nstd*sigma_marg(1)
  ax.set_xlim(pos[0] - s1, pos[0] + s1)
  ax.set_ylim(pos[1] - s2, pos[1] + s2)
  plt.draw()
  return ellip


def lnlike_k(theta, x, y, kcut):
    p1, p2, p3, p4, p5 = theta
    kcond = np.where(x < kcut)
    new_params = np.array([p1, p2, p3, p4, p5])

    cov_mat_red = cov_mat[0: np.array(kcond)[0][-1] + 1, 0: np.array(kcond)[0][-1] + 1]
    icov_red = np.linalg.inv(cov_mat_red)

    debias_factor = (2000.0 - x.shape[0] - 2)/(2000 - 1)
    icov_red = debias_factor * icov_red

    #model = Emu(new_params) ## using emulator predition 
    model, kdum = emu(Omh2=p1, ns=p2, s8=p3, fR0=math.pow(10,p4), n=p5, z=0.0) ## using emulator predition  
    gc.collect()


    x_red = x[kcond]
    y_red = y[kcond]

    
    model_red = model[kcond]
    x_red = x[kcond]
    y_red = y[kcond]
    
    y_model = np.asarray(y_red-model_red)

    loglike = -0.5*(y_model.dot(icov_red).dot(y_model))
    return loglike


dtheta = 5e-3

def partial_derivative(func, var=0, point=[]):
    args = point[:]
    def wraps(x):
        args[var] = x
        return func(*args)
    return derivative(wraps, point[var], dx = dtheta)


def hess_k(ind0, ind1, klim):

    def loglike(p, q):
        params = np.empty_like(fid_params)
        params[:] = fid_params
        params[ind0] = p
        params[ind1] = q

        return lnlike_k(params, kz, pkratioz, klim)

    def Dx(p, q):
        return partial_derivative(func=loglike, var = 0, point=[p, q])

    def Dy(p, q):
        return partial_derivative(func=loglike, var = 1, point=[p, q])

    def Dxy(p, q):
        return partial_derivative(func=Dy, var = 0, point=[p, q])

    def Dyx(p, q):
        return partial_derivative(func=Dx, var = 1, point=[p, q])  

    def Dxx(p, q):
        return partial_derivative(func=Dx, var = 0, point=[p, q]) 

    def Dyy(p, q):
        return partial_derivative(func=Dy, var = 1, point=[p, q]) 


    h = np.zeros(shape = (2,2))
    h[0, 0] = Dxx(fid_params[ind0], fid_params[ind1])
    h[1, 1] = Dyy(fid_params[ind0], fid_params[ind1])
    h[1, 0] = Dyx(fid_params[ind0], fid_params[ind1])
    h[0, 1] = Dxy(fid_params[ind0], fid_params[ind1])
    print(h)
    return h

def params_ell(ind0, ind1):
    params = np.array([fid_params[ind0], fid_params[ind1]])
    return params




In [25]:
gsc=gridspec.GridSpec(5,5, wspace=0.0, hspace=0.0)
fid_params = np.array([Omh2,ns,s8,math.log(fr0,10),nn])
print (fid_params)
num_parameters = 5

[ 0.14174518  0.9667      0.8159     -5.          1.        ]


In [26]:
def lnlike_kgal(theta, x, y, kcut, zz, Cov, shot):
    p1, p2, p3, p4, p5, p6 = theta
    kcond = np.where(x < kcut)
    new_params = np.array([p1, p2, p3, p4, p5, p6])

    cov_mat_red = Cov[0: np.array(kcond)[0][-1] + 1, 0: np.array(kcond)[0][-1] + 1]
    icov_red = np.linalg.inv(cov_mat_red)

    debias_factor = (2000.0 - x.shape[0] - 2)/(2000 - 1)
    icov_red = debias_factor * icov_red

    #model = Emu(new_params) ## using emulator predition 
    #model, kdum = emu(Omh2=p1, ns=p2, s8=p3, fR0=math.pow(10,p4), n=p5, z=0.0) ## using emulator predition 
    #print (p1/h/h, h, p2, p3, math.pow(10,p4), p5, zz, p6, 0)
    model, kdum  =  p_ggmg(p1/h/h, h, p2, p3, math.pow(10,p4), p5, zz, p6, shot)
    gc.collect()
    


    x_red = x[kcond]
    y_red = y[kcond]

    
    model_red = model[kcond]
    x_red = x[kcond]
    y_red = y[kcond]
    
    y_model = np.asarray(y_red-model_red)

    loglike = -0.5*(y_model.dot(icov_red).dot(y_model))
    return loglike

In [27]:
#Define new function for the Loglikehood and Hessian, based on Nesar code, but now for P_gg
def hess_kgal(ind0, ind1, klim, fid_arr, kz, pkratioz, zz, Cov, shot):

    def loglikegal(p, q):
        params = np.empty_like(fid_arr)
        params[:] = fid_arr
        params[ind0] = p
        params[ind1] = q

        return lnlike_kgal(params, kz, pkratioz, klim, zz, Cov, shot)

    def Dx(p, q):
        return partial_derivative(func=loglikegal, var = 0, point=[p, q])

    def Dy(p, q):
        return partial_derivative(func=loglikegal, var = 1, point=[p, q])

    def Dxy(p, q):
        return partial_derivative(func=Dy, var = 0, point=[p, q])

    def Dyx(p, q):
        return partial_derivative(func=Dx, var = 1, point=[p, q])  

    def Dxx(p, q):
        return partial_derivative(func=Dx, var = 0, point=[p, q]) 

    def Dyy(p, q):
        return partial_derivative(func=Dy, var = 1, point=[p, q]) 


    h = np.zeros(shape = (2,2))
    h[0, 0] = Dxx(fid_arr[ind0], fid_arr[ind1])
    h[1, 1] = Dyy(fid_arr[ind0], fid_arr[ind1])
    h[1, 0] = Dyx(fid_arr[ind0], fid_arr[ind1])
    #h[0, 1] = Dxy(fid_arr[ind0], fid_arr[ind1])
    h[0, 1] = h[1, 0]
    #print(h)
    return h

In [29]:
#Loop over various snapshots of the Y10 sample
h=0.6774 # See README and the accompanying paper regarding the value of h. 
Om = 0.3089
Omh2=(h**2)*Om
Obh2 = 0.02203
Och2 = Omh2-Obh2
ns=0.9667
s8=0.8159
### Hu-Sawicki model parameters
#fr0=1e-5
fr0log = -5
n=1
numpar = 6
FY10new = np.zeros([numpar,numpar])
kmax = 0.25
start = time.time()
#for k in (irangey10[8:9]):
for k in (irangey10):    
    astep = timesteps[k,0]
    zstep = timesteps[k,1]
    print (k, zstep)
    #linear bias for galaxy sample, in our case Y10
    b1=b1y10*Dgr(1)/Dgr(astep) #Bias for Y10
    Shot = Veff(zstep, Om, biny10)/Nz(zstep, ay10, z0y10, NY10, biny10)
    #ndens = 1.0/Shot
    param_array = np.array([Omh2,ns,s8,fr0log,n,b1])
    #print (param_array)
    Pfid, kkk = p_ggmg(Om, h, ns, s8, math.pow(10,fr0log), n, zstep, b1, Shot)
    gc.collect()
    covPMGmatz = np.loadtxt('./Covariance_data/covariance_step_'+str(k)+'.txt')
    for i in range(numpar):
     for j in range(numpar):
      #print(i, j)
      if (i > j):
        sub = hess_kgal(j,i, kmax, param_array, kkk, Pfid, zstep, covPMGmatz, Shot)
        FY10new[j,j] += sub[0,0]
        FY10new[i,i] += sub[1,1]
        FY10new[j,i] += sub[0,1]
        FY10new[i,j] += sub[1,0] 
    #np.savetxt('./Covariance_data/FisherHessPgg_step'+str(k)+'.txt', -FY10new)   
elapsed_time_fl = (time.time() - start)
print (elapsed_time_fl)    

43 1.216313
45 1.124045
48 0.999201
53 0.820831
57 0.699524
61 0.593372
65 0.4997
70 0.397038
75 0.307531
82 0.199904
294.0000009536743
